In [2]:
!pip install torchvision

    100% |████████████████████████████████| 61kB 3.4MB/s 
    100% |████████████████████████████████| 519.5MB 29kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5a2d8000 @  0x7f4c2f9e72a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070
    100% |████████████████████████████████| 2.0MB 3.7MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
import torch 
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as data
from torch.autograd import Variable

In [4]:
train_dataset=data.MNIST(root='./data',train=True,transform=transforms.ToTensor(),download=True)

Processing...
Done!


In [5]:
(train_dataset[0][0].size())

torch.Size([1, 28, 28])

In [0]:
test_dataset=data.MNIST(root='./data',train=False,transform=transforms.ToTensor())

In [0]:
batch_size=100

In [0]:
n_iters = 3000 

In [0]:
num_epochs=n_iters/(len(train_dataset)/batch_size)

In [0]:
num_epochs=int(num_epochs)

In [0]:
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

In [0]:
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

In [0]:
class LSTMModel(nn.Module):
  def __init__(self,input_dim,hidden_dim,layer_dim,output_dim):
    super(LSTMModel,self).__init__()
    self.hidden_dim=hidden_dim
    self.layer_dim=layer_dim
    self.lstm=nn.LSTM(input_dim,hidden_dim,layer_dim,batch_first=True)
    self.linear=nn.Linear(hidden_dim,output_dim)
  def forward(self,x):
    h0=Variable(torch.zeros(self.layer_dim,x.size(0),self.hidden_dim))
    c0=Variable(torch.zeros(self.layer_dim,x.size(0),self.hidden_dim)) 
    out,(hn,c0)=self.lstm(x,(h0,c0))
    #getting the last layer
    out =self.linear(out[:,-1,:])
    return out

In [0]:
input_dim=28
hidden_dim=100
layer_dim=1
output_dim=10

In [0]:
 model=LSTMModel(input_dim,hidden_dim,layer_dim,output_dim)

In [0]:
criterion=nn.CrossEntropyLoss()

In [0]:
learning_rate=0.01
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

In [18]:
len(list(model.parameters()))

6

In [0]:
seq_dim=28

iter=0

for epoch in range(num_epochs):
  for i,(images,labels) in enumerate (train_loader):
    images=Variable(images.view(-1,seq_dim,input_dim))
    labels=Variable(labels)
    optimizer.zero_grad()
    outputs=model(images)
    loss=criterion(outputs,labels)
    loss.backward()
    optimizer.step()
    iter=iter+1
    if(iter%500==0):
      correct=0
      total=0
      for images,labels in test_loader:
        images=Variable(images.view(-1,seq_dim,input_dim))
        outputs=model(images)
        _,predicted=torch.max(outputs.data,1)
        total=total+labels.size(0)
        correct=correct+(predicted==labels).sum()
        accuracy=float (100*correct/total)
      print("Iteration {}. Loss {}. Accuracy {:{}f} ".format(iter,loss.data[0],accuracy,0.2))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Iteration 500. Loss 0.8163042664527893. Accuracy 75.00 
Iteration 1000. Loss 0.7232294678688049. Accuracy 80.00 
Iteration 1500. Loss 0.4400777816772461. Accuracy 84.00 
Iteration 2000. Loss 0.43462470173835754. Accuracy 86.00 
Iteration 2500. Loss 0.3732433617115021. Accuracy 88.00 
